In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-26"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11518,2024-02-26,Europa Eurobasket,15:30,Países Baixos,Grécia,3.14,1.34,151.5,1.80,1.92,0.0,0.00,0.00,r9yAB48p,0.318471,0.746269,0.555556,0.520833,0.064740,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,94,72,2.99,1.20,0.000,0.000,0.568211,0.045620,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11519,2024-02-26,Europa Eurobasket,15:00,Bósnia e Herzegovina,França,3.95,1.23,156.5,1.83,1.91,0.0,0.00,0.00,CQpvHQpN,0.253165,0.813008,0.546448,0.523560,0.066173,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,99,73,1.04,1.11,0.000,0.000,0.742599,0.030251,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11520,2024-02-26,Europa Eurobasket,14:00,Polónia,Macedônia do Norte,1.09,6.74,150.5,1.84,1.90,0.0,0.00,0.00,x8I0YRFc,0.917431,0.148368,0.543478,0.526316,0.065799,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,64,69,3.14,2.60,0.000,0.000,1.020473,0.022688,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11521,2024-02-26,Europa Eurobasket,14:00,Geórgia,Sérvia,3.53,1.28,158.5,1.85,1.89,0.0,0.00,0.00,bHJdZ70i,0.283286,0.781250,0.540541,0.529101,0.064536,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,69,77,1.59,1.04,0.000,0.000,0.661534,0.015125,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11522,2024-02-26,Eua Ncaa,23:00,TCU,Baylor,1.73,2.18,148.5,1.83,1.83,0.0,0.00,0.00,dnO4hLv7,0.578035,0.458716,0.546448,0.546448,0.036750,151.748,54.255548,0.357537,1.8,1.643168,0.912871,108.75,2.100,0.874871,0.416605,24.0,155.544,44.634774,0.286959,1.2,1.643168,1.369306,159.60,2.146,0.953929,0.444515,14.0,75,76,1.45,2.10,108.758,155.278,0.162761,0.000000,NaN,-0.93,-0.186,-3.924731,0.641766,0.6,-0.041766,-1.99,-0.398,-2.964824,0.589019,0.4,-0.189019
11523,2024-02-26,Europa Eurobasket,13:30,Finlândia,Dinamarca,1.34,3.14,153.5,1.83,1.91,0.0,0.00,0.00,MazEAONj,0.746269,0.318471,0.546448,0.523560,0.064740,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,61,75,9.35,2.30,0.000,0.000,0.568211,0.030251,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11524,2024-02-26,Eua Ncaa,22:30,Miss. Valley St.,Prairie View A&M,3.97,1.23,135.5,1.87,1.95,0.0,0.00,0.00,f1G0KcEi,0.251889,0.813008,0.534759,0.512821,0.064897,466.940,111.037192,0.237798,0.0,0.000000,NaN,332.80,8.560,2.836018,0.331311,-84.0,142.588,31.114345,0.218212,0.6,1.341641,2.236068,120.95,2.066,0.252052,0.122000,-31.0,52,59,6.40,2.05,383.534,170.378,0.745182,0.029617,NaN,-5.00,-1.000,-2.970000,0.000000,0.0,0.000000,0.34,0.068,3.382353,0.369506,0.2,-0.169506
11525,2024-02-26,Eua Ncaa,22:30,Campbell,NC Wilmington,3.47,1.30,143.5,1.80,1.95,0.0,0.00,0.00,pCfRVai5,0.288184,0.769231,0.555556,0.512821,0.057415,227.110,119

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
